# Replicate Fama-French Characteristic-sorted portfolios and Factors

Rong Wang, July 2025

In [1]:
# Packages 
import numpy as np
import pandas as pd
import os
import gc
import time
import datetime as dt
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import *
from pathlib import Path
from tqdm import tqdm
import wrds
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

--- 
## Data Preparation

### Load JKP factors

Focusing on FF-equivalent characteristics: me, be_me, ope_be, at_gr1, ret_12_1.

In [ ]:
# 5 factors from JKP
jkp = pd.read_csv('../Data/JKP_factors_monthly.csv') 
jkp = jkp.pivot(index='date', columns='name', values='ret').reset_index() # long to wide
jkp = jkp[['date','market_equity','be_me','ope_be','at_gr1','ret_12_1']]
jkp['date'] = pd.to_datetime(jkp['date']).dt.year * 100 + pd.to_datetime(jkp['date']).dt.month
jkp

name,date,market_equity,be_me,ope_be,at_gr1,ret_12_1
0,192601,0.027025,NaN,NaN,NaN,NaN
1,192602,-0.029599,NaN,NaN,NaN,NaN
2,192603,-0.051052,NaN,NaN,NaN,NaN
3,192604,0.005405,NaN,NaN,NaN,NaN
4,192605,-0.005611,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1183,202408,-0.050941,-0.020158,0.012547,-0.009432,0.029552
1184,202409,-0.018374,-0.019773,-0.006242,0.003045,0.010313
1185,202410,0.001539,-0.004121,-0.016460,-0.015150,0.024211
1186,202411,0.019527,-0.015056,-0.033224,-0.029310,0.031126


Check factor correlations between FF and JKP.

In [3]:
# 6 factors from Fama-French
ff6 = pd.read_csv('../Data/raw/FF6_factors_monthly.csv')
ff6['MKT'] = ff6['Mkt-RF'] + ff6['RF']
ff6 = ff6.drop(columns=['Mkt-RF','RF'])
ff6.loc[:, ff6.columns != 'date'] = ff6.loc[:, ff6.columns != 'date'] / 100 # remove percentage
ff6

,date,SMB,HML,RMW,CMA,MOM,MKT
0,196307,-0.0048,-0.0081,0.0064,-0.0115,0.0101,-0.0012
1,196308,-0.0080,0.0170,0.0040,-0.0038,0.0100,0.0533
2,196309,-0.0043,0.0000,-0.0078,0.0015,0.0012,-0.0130
3,196310,-0.0134,-0.0004,0.0279,-0.0225,0.0313,0.0283
4,196311,-0.0085,0.0173,-0.0043,0.0227,-0.0078,-0.0059
...,...,...,...,...,...,...,...
733,202408,-0.0355,-0.0110,0.0075,0.0082,0.0481,0.0209
734,202409,-0.0092,-0.0277,0.0018,-0.0029,-0.0062,0.0213
735,202410,-0.0088,0.0086,-0.0142,0.0098,0.0296,-0.0061
736,202411,0.0460,0.0015,-0.0230,-0.0205,0.0101,0.0689


In [4]:
print(ff6.set_index('date')['SMB'].corr(jkp.set_index('date')['market_equity']))
print(ff6.set_index('date')['HML'].corr(jkp.set_index('date')['be_me']))
print(ff6.set_index('date')['RMW'].corr(jkp.set_index('date')['ope_be']))
print(ff6.set_index('date')['CMA'].corr(jkp.set_index('date')['at_gr1']))
print(ff6.set_index('date')['MOM'].corr(jkp.set_index('date')['ret_12_1']))

0.8949390353858966
0.8165582512483179
0.8729767803936969
0.7643008553547642
0.9544420608473749


### Individual Stock Returns and Characteristics

In [5]:
%%time
# Load stock returns and characteristics data
charc = pd.read_csv('/work/rw196/data/JKP_stock_charcs.csv')
charc = charc.rename(columns={"eom": "date"})
charc['date'] = pd.to_datetime(charc['date'])+MonthEnd(0)
charc = charc[['date','permno','me','size_grp','ret','market_equity','be_me','ope_be','at_gr1','ret_12_1']]
charc = charc.dropna(subset=['ret','permno']).sort_values(by=['permno','date']).reset_index(drop=True)
charc

CPU times: user 1min 39s, sys: 9.31 s, total: 1min 49s
Wall time: 1min 50s


,date,permno,me,size_grp,ret,market_equity,be_me,ope_be,at_gr1,ret_12_1
0,1986-02-28,10000.0,1.196000e+01,micro,-0.257143,1.196000e+01,NaN,NaN,NaN,NaN
1,1986-03-31,10000.0,1.633000e+01,micro,0.365385,1.633000e+01,NaN,NaN,NaN,NaN
2,1986-04-30,10000.0,1.517200e+01,micro,-0.098592,1.517200e+01,NaN,NaN,NaN,NaN
3,1986-05-31,10000.0,1.179386e+01,nano,-0.222656,1.179386e+01,0.058420,-0.763425,NaN,NaN
4,1986-06-30,10000.0,1.173459e+01,nano,-0.005025,1.173459e+01,0.058715,-0.763425,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3660022,2024-08-31,93436.0,6.840044e+05,mega,-0.077390,6.840044e+05,0.094119,0.187362,0.257886,-0.100783
3660023,2024-09-30,93436.0,8.390474e+05,mega,0.221942,8.390474e+05,0.076727,0.187362,0.257886,-0.144313
3660024,2024-10-31,93436.0,8.020335e+05,mega,-0.045025,8.020335e+05,0.082874,0.179876,0.245510,0.302679
3660025,2024-11-30,93436.0,1.107984e+06,mega,0.381469,1.107984e+06,0.059990,0.179876,0.245510,0.040695


In [6]:
charc_list = ['market_equity','be_me','ope_be','at_gr1','ret_12_1']
len(charc_list)

5

## Construct Portfolios, Factors, and Weights

Calculates breakpoints to divide data into a specified number of equal-sized groups.

In [7]:
def get_equal_group_breakpoints(arr, num_groups: int):
    """
    This function determines the values that split a dataset into `num_groups`
    portfolios, each containing an equal number of observations.

    Args:
        arr (np.ndarray): The input 1D array of data. NaNs are ignored.
        num_groups (int): The number of equal-sized groups to create (e.g., 3 for terciles).

    Returns:
        np.ndarray: An array of the calculated breakpoints. The number of breakpoints
                    will be `num_groups` - 1.
    """
    # Remove NaNs
    clean_arr = arr[~np.isnan(arr)]
        
    # If the array is empty after cleaning, return NaNs for the breakpoints
    if len(clean_arr) == 0:
        return np.full(num_groups - 1, np.nan)

    # Sort the data
    sorted_arr = np.sort(clean_arr)
    n = len(sorted_arr)

    # Determine the desired percentiles for the breakpoints
    #    For `num_groups = 3`, we need percentiles at [33.33, 66.67]
    #    For `num_groups = 5`, we need percentiles at [20, 40, 60, 80]
    p_desired = 100 * np.arange(1, num_groups) / num_groups

    # Create the percentile ranks corresponding to each data point
    #    It considers the k-th value to be the 100*(k-0.5)/n percentile (same as Matlab convention)
    p_rank = 100 * (np.arange(1, n + 1)-0.5) / n

    # Use linear interpolation to find the data values at the desired percentiles.
    #    This finds the breakpoints.
    breakpoints = np.interp(p_desired, p_rank, sorted_arr)

    return breakpoints

Calculates the weighted average of returns and the corresponding weights.

In [8]:
def getPtfRetsWts(df: pd.DataFrame, ret_name: str, me_name: str, method: str) -> tuple[float, pd.Series]:
    """
    This function computes portfolio returns and weights using one of three schemes:
    'ew' for equal-weighted, 'vw' for value-weighted, and 'cvw' for
    capped value-weighted.

    Args:
        df (pd.DataFrame): DataFrame containing stock data.
        ret_name (str): The name of the column containing stock returns.
        me_name (str): The name of the column containing market equity values.
        method (str): The weighting method ('vw', 'ew', or 'cvw').

    Returns:
        A tuple containing:
        - float: The calculated portfolio return for the given period.
        - pd.Series: The weights for each stock in the portfolio.

    Raises:
        ValueError: If an invalid method is specified or for 'cvw' issues.
    """
    # At least 5 stocks in each of the long and short legs
    if len(df) < 5:
        return np.nan, pd.Series(dtype=float)
        
    rt = df[ret_name]
    me = df[me_name]
    
    if method == 'ew':
        # Equal weights for all stocks
        weights = pd.Series(1 / len(df), index=df.index)
        portfolio_return = rt.mean()

    elif method == 'vw':
        # Value weights based on market equity
        total_me = me.sum()
        weights = me / total_me
        portfolio_return = (rt * weights).sum()

    elif method == 'cvw':
        if 'size_grp' not in df.columns:
            raise ValueError("Data does not have size_grp indicators required for 'cvw' method.")
        
        # Cap market equity at the 80th percentile of NYSE stocks
        nyse80 = df['size_grp'] == 'mega'
        cap = me[nyse80].min() if np.any(nyse80) else np.nan  
        capped_me = me.clip(upper=cap)
        
        # Calculate weights based on capped market equity
        total_capped_me = capped_me.sum()
        weights = capped_me / total_capped_me
        portfolio_return = (rt * weights).sum()

    else:
        raise ValueError("Method must be one of 'vw', 'ew', or 'cvw'.")
        
    return portfolio_return, weights

In [9]:
# Portfolio sorts are based on non-micro stocks (i.e., larger than NYSE 20th percentile)
non_micro = charc[charc['size_grp'].isin(['mega', 'large','small'])]

In [11]:
%%time
# Store results
returns_list = []
weights_list = []

# Loop through the charcs
Tbar = tqdm(charc_list, colour='green', desc='Processing')
for ch in Tbar:
    Tbar.set_postfix({'Charc': ch})
     
    # --- Assign portfolios ---
    
    # Sort non-micro stocks into three groups of equal numbers
    breaks = non_micro.groupby(['date'])[ch].apply(lambda x: get_equal_group_breakpoints(x.values,3)).apply(pd.Series).rename(columns={0: f'{ch}1', 1: f'{ch}2'}).reset_index()
    
    # Merge breakpoints to the original data
    breaks = pd.merge(charc, breaks, how='left', on=['date'])
    
    # Assign portfolio buckets
    ptfassign = breaks[['date','permno']].copy()
    conditions = [
        breaks[ch]<=breaks[f'{ch}1'],
        (breaks[ch]>breaks[f'{ch}1']) & (breaks[ch]<=breaks[f'{ch}2']),
        breaks[ch]>breaks[f'{ch}2']
    ]
    choices = [1,2,3] # list(range(1, 4))
    ptfassign[ch] = np.select(conditions, choices, default=np.nan)
    
    # Move date 1M forward such that ptf assignments are 1M lagged 
    ptfassign['date'] = ptfassign['date'] + pd.DateOffset(months=1) + MonthEnd(0)
    
    # Merge back with monthly records
    ptfassign = pd.merge(charc[['date','permno','me','ret','size_grp']], ptfassign, how='left', on=['date','permno'])
    
    # Lagged mkt cap
    ptfassign['lme'] = ptfassign.sort_values(by=['permno','date']).groupby(['permno'])['me'].shift(1)
    
    # --- Portfolio returns and weights ---

    # Focus on the top and bottom decile portfolios
    ptfassign = ptfassign[ptfassign[ch]!=2]

    # Group the data (dropping missing values)
    groups = ptfassign.dropna(subset=['ret', 'lme', ch]).groupby(['date', ch])
    
    # Loop through groups to get both returns and weights
    for group_keys, group_df in groups:
        dt, ptfid = group_keys
        group_df = group_df.set_index('permno')
        
        # Call your function once per group
        ptfret, ptfwts = getPtfRetsWts(group_df, 'ret', 'lme', 'cvw')
        
        # Append the aggregated return for the group
        returns_list.append({
            'date': dt,
            'ptfid': f'{ch}_{ptfid.astype(int)}',
            'vwret': ptfret
        })
        
        # Append the detailed weights Series for all stocks in the group
        for permno, wt in ptfwts.items():
            weights_list.append({
                'date': dt,
                'ptfid': f'{ch}_{ptfid.astype(int)}',
                'permno': permno,
                'wts': wt
            })
    
# Create the DataFrames
port = pd.DataFrame(returns_list)
port_wts = pd.DataFrame(weights_list)     

Processing: 100%|██████████| 5/5 [01:10<00:00, 14.16s/it]


CPU times: user 1min 22s, sys: 1.76 s, total: 1min 24s
Wall time: 1min 24s


In [12]:
port

,date,ptfid,vwret
0,1960-01-31,market_equity_1,-0.032317
1,1960-01-31,market_equity_3,-0.056882
2,1960-02-29,market_equity_1,0.011833
3,1960-02-29,market_equity_3,0.012073
4,1960-03-31,market_equity_1,-0.032977
...,...,...,...
7795,2024-10-31,ret_12_1_3,0.003371
7796,2024-11-30,ret_12_1_1,0.075006
7797,2024-11-30,ret_12_1_3,0.105958
7798,2024-12-31,ret_12_1_1,-0.056122


In [13]:
port_wts

,date,ptfid,permno,wts
0,1960-01-31,market_equity_1,10014.0,0.000774
1,1960-01-31,market_equity_1,10022.0,0.001665
2,1960-01-31,market_equity_1,10057.0,0.002132
3,1960-01-31,market_equity_1,10188.0,0.004413
4,1960-01-31,market_equity_1,10217.0,0.002301
...,...,...,...,...
12284659,2024-12-31,ret_12_1_3,93340.0,0.000192
12284660,2024-12-31,ret_12_1_3,93368.0,0.000022
12284661,2024-12-31,ret_12_1_3,93371.0,0.000095
12284662,2024-12-31,ret_12_1_3,93397.0,0.000080


In [14]:
# Reshape
port_wide = port.pivot(index='date', columns='ptfid', values='vwret').reset_index()
port_wide.columns.name = None
port_wide

,date,at_gr1_1,at_gr1_3,be_me_1,be_me_3,market_equity_1,market_equity_3,ope_be_1,ope_be_3,ret_12_1_1,ret_12_1_3
0,1960-01-31,-0.058072,-0.055743,-0.069151,-0.053001,-0.032317,-0.056882,-0.058291,-0.055848,-0.043787,-0.070642
1,1960-02-29,-0.001771,0.017512,0.023800,-0.009397,0.011833,0.012073,-0.016082,0.018887,-0.009006,0.027680
2,1960-03-31,-0.023446,-0.013622,-0.007444,-0.046512,-0.032977,-0.014057,-0.037050,-0.015128,-0.021069,-0.016976
3,1960-04-30,-0.012292,-0.003232,-0.001030,-0.015551,-0.023981,-0.009502,-0.025732,0.005812,-0.023930,-0.008095
4,1960-05-31,0.019288,0.052490,0.066043,0.013126,0.027724,0.034795,0.009375,0.052500,0.009732,0.053689
...,...,...,...,...,...,...,...,...,...,...,...
775,2024-08-31,0.004583,0.013796,0.022039,0.002091,-0.028883,0.022354,0.001924,0.014367,-0.006117,0.023402
776,2024-09-30,0.020501,0.017286,0.028184,0.008362,0.003290,0.021232,0.024097,0.017739,0.014031,0.024125
777,2024-10-31,-0.014700,0.000433,-0.002346,-0.006463,-0.007030,-0.008483,0.003091,-0.013565,-0.020433,0.003371
778,2024-11-30,0.075581,0.104144,0.094610,0.078615,0.096765,0.076713,0.106387,0.072355,0.075006,0.105958


## Factors

In [15]:
%%time
psf = port_wide[['date']]
for ch in tqdm(charc_list, desc='Processing', colour='green'):
    if port_wide[f'{ch}_1'].mean() >= port_wide[f'{ch}_3'].mean():
        psf[ch] = port_wide[f'{ch}_1'] - port_wide[f'{ch}_3']
    else:
        psf[ch] = port_wide[f'{ch}_3'] - port_wide[f'{ch}_1']
psf['date'] = pd.to_datetime(psf['date']).dt.year * 100 + pd.to_datetime(psf['date']).dt.month
psf

Processing: 100%|██████████| 5/5 [00:00<00:00, 1360.20it/s]

CPU times: user 10 ms, sys: 3.03 ms, total: 13.1 ms
Wall time: 11.5 ms


,date,market_equity,be_me,ope_be,at_gr1,ret_12_1
0,196001,0.024564,0.016150,0.002443,-0.002328,-0.026855
1,196002,-0.000241,-0.033197,0.034969,-0.019283,0.036686
2,196003,-0.018920,-0.039067,0.021922,-0.009825,0.004093
3,196004,-0.014479,-0.014521,0.031544,-0.009060,0.015835
4,196005,-0.007071,-0.052917,0.043125,-0.033202,0.043958
...,...,...,...,...,...,...
775,202408,-0.051237,-0.019948,0.012444,-0.009213,0.029519
776,202409,-0.017942,-0.019822,-0.006358,0.003215,0.010093
777,202410,0.001453,-0.004116,-0.016656,-0.015132,0.023804
778,202411,0.020051,-0.015995,-0.034033,-0.028563,0.030952


In [16]:
for ch in charc_list:
    print(jkp.set_index('date')[ch].corr(psf.set_index('date')[ch]))

0.9978048359826764
0.996746834664578
0.9932570921834325
0.9926641139443204
0.9974601482582877
